# Regression 02 : AirQuality

#### 머신러닝 코드 구조 <br>

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")



# [3] 새로운 알고리즘 KNN(k-Nearst Neighber) + Scaling

새로운 알고리즘을 적용하는데 알아야 할 것은 다음의 5가지 입니다.
1. 알고리즘 원리
2. 전제조건
3. 문법(함수 명 등)
4. 성능 튜닝 포인트(Hyper Parameter 튜닝)
    - 복잡도를 결정하는 요소(4번과 밀접한 관련이 있음)

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
data_path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/airquality_simple.csv'
data = pd.read_csv(data_path)

## 20.데이터 준비

### 21.변수 정리

### 22.NA 처리

### 23.Feature Engineering

* 날짜 변수 만들기  
airquality 데이터는 1973년 5~9월까지 데이터 입니다.  
month, day 데이터를 날짜 형식으로 변환해 봅시다.

(중요!)날짜변수를 만드는 이유!  
* 날짜로 부터 추가 변수를 도출해 내기 위해서 날짜 변수를 만듭니다.  
* 만약 추가변수 도출이 끝나면, 날짜변수는 제거합니다.

In [3]:
# 날짜를 yyyymmdd로 만든 후 변환해봅시다.
data['Date'] = pd.to_datetime(1973*10000 + data.Month*100 + data.Day
                              ,format='%Y%m%d')

# 요일을 추가해 봅시다. 
data['WeekDay'] = data.Date.dt.dayofweek

# working day 여부
data['WorkingDay'] = 1
data.loc[data['WeekDay'].isin([0,6]), 'WorkingDay'] = 0

# 전날 오존 농도를 추가합니다.
data['Ozone_lag1'] = data['Ozone'].shift() 

# 전날 온도, 전날 바람세기, 전날 solar.R
data['Temp_lag1'] = data['Temp'].shift() 
data['Wind_lag1'] = data['Wind'].shift() 
data['Solar.R_lag1'] = data['Solar.R'].shift() 

# 전전날 오존농도 대비 전날 오존농도 증감
data['Ozone_diff'] = data['Ozone_lag1'] -  data['Ozone'].shift(2)
data.head()

# 7일 이동평균 오존농도
data['Ozone_MA_7_lag1'] = data['Ozone'].rolling(7, min_periods=1).mean().shift()

# 3일 이동평균 온도를 추가합시다.
data['Temp_MA_3_lag1'] = data['Temp'].rolling(3, min_periods=1).mean().shift()
data.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WorkingDay,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lag1
0,41,190.0,7.4,67,5,1,1973-05-01,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,118.0,8.0,72,5,2,1973-05-02,2,1,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000
2,12,149.0,12.6,74,5,3,1973-05-03,3,1,36.0,72.0,8.0,118.0,-5.0,38.500000,69.500000
3,18,313.0,11.5,62,5,4,1973-05-04,4,1,12.0,74.0,12.6,149.0,-24.0,29.666667,71.000000
4,19,NaN,14.3,56,5,5,1973-05-05,5,1,18.0,62.0,11.5,313.0,6.0,26.750000,69.333333


### 24.Dummy Variable
    * 범주형 변수를 숫자로 만드는 방법
    * pd.get_dummies, pd.concat, (Pandas Dataframe).drop
    * 불필요한 칼럼들 제거


In [4]:
data_wd = pd.get_dummies(data['WeekDay'], prefix = 'W', drop_first = 1)
data_wd.head()

,W_1,W_2,W_3,W_4,W_5,W_6
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
4,0,0,0,0,1,0


In [5]:
data2 = pd.concat([data, data_wd], axis=1)
data2.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WorkingDay,Ozone_lag1,...,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lag1,W_1,W_2,W_3,W_4,W_5,W_6
0,41,190.0,7.4,67,5,1,1973-05-01,1,1,NaN,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
1,36,118.0,8.0,72,5,2,1973-05-02,2,1,41.0,...,190.0,NaN,41.000000,67.000000,0,1,0,0,0,0
2,12,149.0,12.6,74,5,3,1973-05-03,3,1,36.0,...,118.0,-5.0,38.500000,69.500000,0,0,1,0,0,0
3,18,313.0,11.5,62,5,4,1973-05-04,4,1,12.0,...,149.0,-24.0,29.666667,71.000000,0,0,0,1,0,0
4,19,NaN,14.3,56,5,5,1973-05-05,5,1,18.0,...,313.0,6.0,26.750000,69.333333,0,0,0,0,1,0


In [6]:
drop_x = ['Solar.R','Wind','Temp','Month','Day','Date','WeekDay']
data2.drop(drop_x, axis = 1, inplace = True)
data2.head()

,Ozone,WorkingDay,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lag1,W_1,W_2,W_3,W_4,W_5,W_6
0,41,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0
1,36,1,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000,0,1,0,0,0,0
2,12,1,36.0,72.0,8.0,118.0,-5.0,38.500000,69.500000,0,0,1,0,0,0
3,18,1,12.0,74.0,12.6,149.0,-24.0,29.666667,71.000000,0,0,0,1,0,0
4,19,1,18.0,62.0,11.5,313.0,6.0,26.750000,69.333333,0,0,0,0,1,0


In [ ]:
# dummy variable
data_wd =      (    ,    ,    )

In [ ]:
data2 = pd.     ([   ,    ], axis=1)

* 불필요한 변수를 제거합시다.

In [ ]:
# 칼럼삭제
drop_x = [         ]
data2.    (drop_x, axis = 1, inplace = True)

### 25.Data Split

데이터셋을 나눠봅시다.  
위 코드를 참조해서 직접 코드를 작성합시다. 


In [54]:
# 필요한 함수를 불러옵니다.
from sklearn.model_selection import train_test_split

In [55]:
# features와 target 분리
X = data2.drop('Ozone', axis=1)# == data.iloc[:, 1:]
y = data.iloc[:, 0]

In [56]:
# 전체에서 train : test = 7 : 3
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

### (Again)22.NA 처리 : knn Imputation
(아래 내용은 이번 과정에서는 설명을 하지 않고 그냥 실행하고 넘어갑시다.)

In [57]:
train_x.isnull().sum()
# train_x[train_x['Solar.R_lag1'].isna()].head()

WorkingDay         0
Ozone_lag1         1
Temp_lag1          1
Wind_lag1          1
Solar.R_lag1       6
Ozone_diff         2
Ozone_MA_7_lag1    1
Temp_MA_3_lag1     1
W_1                0
W_2                0
W_3                0
W_4                0
W_5                0
W_6                0
dtype: int64

In [58]:
test_x.isnull().sum()
# test_x[test_x['Solar.R_lag1'].isna()].head()

WorkingDay         0
Ozone_lag1         0
Temp_lag1          0
Wind_lag1          0
Solar.R_lag1       2
Ozone_diff         0
Ozone_MA_7_lag1    0
Temp_MA_3_lag1     0
W_1                0
W_2                0
W_3                0
W_4                0
W_5                0
W_6                0
dtype: int64

In [59]:
# 필요한 함수 불러오기
from sklearn.impute import KNNImputer

# 선언한다.
imputer = KNNImputer()

# 함수를 만든다.
imputer.fit(train_x)

# 적용한다.
train_x = imputer.transform(train_x)
test_x = imputer.transform(test_x)

In [60]:
type(train_x)

numpy.ndarray

In [63]:
print(np.isnan(train_x).sum(axis=0))
print(np.isnan(test_x).sum(axis=0))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [52]:
# train_x[:,0]
train_x

array([[0.        , 0.11976048, 0.45      , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.05389222, 0.4       , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.02994012, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [1.        , 0.26946108, 0.625     , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.10179641, 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.1497006 , 0.55      , ..., 0.        , 0.        ,
        0.        ]])

### 26.Scaling features
knn 알고리즘으로 모델링 하기 위해서는 반드시 scaling이 필요!



In [27]:
# 필요한 함수 로딩
from sklearn.preprocessing import MinMaxScaler

In [28]:
# 함수 선언
scaler = MinMaxScaler()

# 함수 만들기
scaler.fit(train_x)

MinMaxScaler()

In [64]:
# scaling 전
print(train_x.min(axis=0).round(2))
print(train_x.max(axis=0))

[  0.     1.    57.     3.4    7.   -95.    13.43  57.33   0.     0.
   0.     0.     0.     0.  ]
[  1.         168.          97.          20.7        332.
 123.          98.71428571  94.66666667   1.           1.
   1.           1.           1.           1.        ]


In [65]:
# 함수 적용하기
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [66]:
# scaling 후
print(train_x.min(axis=0))
print(train_x.max(axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### 27.Dataframe to Numpy array

In [67]:
type(train_x)

numpy.ndarray

sk-learn의 transform 함수를 통해 변환된 결과물은 numpy array가 됩니다.  
그러므로 다시 할 필요가 없습니다. 

## 30.모델링 : knn

### 31.import

[K-Nearest Neighbors Regression](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor)

검색요령 : sklearn <알고리즘> 회귀?분류?  
예 : knn 회귀 ==> sklearn knn regressssor  
svm 분류 ==> sklearn svm classifier



In [68]:
# 검색창에, sklearn knn regressor 로 검색합시다.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

### 32.모델선언

k 값에 따라 성능은 달라집니다. 하지만 여기서는 default(5)로 두고 모델링 합니다.

In [69]:
model_knn =  KNeighborsRegressor(5)

### 33.모델링(학습)

In [71]:
model_knn.fit(train_x, train_y)

KNeighborsRegressor()

### 34.예측

In [72]:
test_pred = model_knn.predict(test_x)

### 35.평가

In [73]:
#test set에서의 성능 확인
# MSE, 
print(mean_squared_error(test_y, test_pred))

# RMSE
print(mean_squared_error(test_y, test_pred, squared=0))#squared 가 제곱의 의미기에 루트가 제곱을 제거하는 것이기에 0으로 설정하는 것이다.

# MAE
print(mean_absolute_error(test_y  , test_pred))

760.0147826086957
27.5683656136648
19.89130434782609


Linear Regression 과 knn 어떤 알고리즘으로 모델링한 결과가 더 나은가요?


In [75]:
from sklearn.linear_model import LinearRegression

model_lm = LinearRegression()
model_lm.fit(train_x, train_y)
test_pred_lm = model_lm.predict(test_x)
print(mean_squared_error(test_y, test_pred_lm, squared=0))

24.489611047536446


In [79]:
for k in range(3, 21):
    model_knn_n = KNeighborsRegressor(n_neighbors=k)
    model_knn_n.fit(train_x, train_y)
    test_pred_knn_n = model_knn_n.predict(test_x)
    print('k:{}, RMSE : {}'.format(k, mean_squared_error(test_y, test_pred_knn_n, squared=0)))

k:3, RMSE : 32.599494676146385
k:4, RMSE : 31.041171817862917
k:5, RMSE : 27.5683656136648
k:6, RMSE : 27.517572339136557
k:7, RMSE : 27.402282626958325
k:8, RMSE : 28.301967399734814
k:9, RMSE : 28.97110189542141
k:10, RMSE : 29.66137145382423
k:11, RMSE : 30.3788452123431
k:12, RMSE : 30.715661181023595
k:13, RMSE : 30.304082344469975
k:14, RMSE : 29.719003447747713
k:15, RMSE : 29.818875129851257
k:16, RMSE : 29.23153565296279
k:17, RMSE : 29.136092997340803
k:18, RMSE : 29.103386610232295
k:19, RMSE : 27.8342139500477
k:20, RMSE : 27.693054575816124
